In [1]:
from core.board import Board
from analysis.msm_builder import get_msm_graph
from solver.solver import solve

In [2]:
r = 16
c = 30
n = 99
start = (0,0)
seed = None          
preset = False
profile = False
B_t = Board.random_complete(r,c,n,exclude=[start],seed=seed)
B_k = Board(r,c,n)
if preset:
    B_k.set_clear(0,0,1)
    B_k.set_clear(1,2,1)
    B_k.set_clear(2,1,1)
if preset: print(B_k)
elif seed is not None: print(B_t)

In [3]:
from analysis.msm_analysis import find_solutions, seperate_connected_msm_components
from analysis.probability import calculate_probabilities
if preset:
    B_o, msm, c, f = get_msm_graph(B_k, order=2, verbose=0)
    print(B_o, msm, c, f)
    msms = seperate_connected_msm_components(msm)
    print(find_solutions(msms[0], verbose=0))
    print(calculate_probabilities(B_k, verbose=0))

In [4]:
import cProfile
if not preset and seed is not None:
    B_k = Board(r,c,n)
    if profile:
        cProfile.run('print(solve(B_k, B_t, first_move=start, verbose=0))')
    else:
        print(solve(B_k, B_t, first_move=start, verbose=2))

In [5]:
import random
if seed is None and not preset:
    start = (0,0)
    max_iterations = 1000
    max_wins = 200
    wins = 0
    win_time = 0
    win_guesses = 0
    win_risk = 0
    ng_games = 0
    i = 0
    for i in range(max_iterations):
        seed = random.randint(0, 2**32)
        B_t = Board.random_complete(r,c,n,exclude=[start], seed=seed)
        B_k = Board(r,c,n)
        print(seed, end=' ')
        win, stats = solve(B_k, B_t, first_move=start, verbose=0)
        if win: 
            wins += 1
            win_time += stats.time
            win_guesses += stats.guesses
            win_risk += stats.risk
            if stats.guesses == 0: ng_games += 1
        print(win, f'{wins}/{i+1}', stats)
        if wins >= max_wins: break
    seed = None
    i += 1
    print(f'Wins: {wins/i}, Avg_Time: {win_time/wins}, Avg_guess: {win_guesses/wins}, Avg_risk:{win_risk/wins}, NG: {ng_games}')

127034522 False 0/1 [Time: 1.210, Eff: 8%, Risk: 50.00%, Guesses: 1, Uncaught Logic: 0]
2688514668 True 1/2 [Time: 3.478, Eff: 3%, Risk: 89.16%, Guesses: 12, Uncaught Logic: 2]
3798294685 False 1/3 [Time: 1.062, Eff: 34%, Risk: 37.28%, Guesses: 4, Uncaught Logic: 0]
2838364876 False 1/4 [Time: 0.319, Eff: 76%, Risk: 49.26%, Guesses: 5, Uncaught Logic: 0]
3771921741 False 1/5 [Time: 1.118, Eff: 6%, Risk: 76.48%, Guesses: 4, Uncaught Logic: 0]
1121469407 True 2/6 [Time: 2.870, Eff: 6%, Risk: 50.00%, Guesses: 1, Uncaught Logic: 2]
2939689750 False 2/7 [Time: 0.165, Eff: 160%, Risk: 63.20%, Guesses: 5, Uncaught Logic: 0]
1054196087 True 3/8 [Time: 1.899, Eff: 4%, Risk: 76.92%, Guesses: 5, Uncaught Logic: 0]
463548041 False 3/9 [Time: 1.738, Eff: 6%, Risk: 64.83%, Guesses: 3, Uncaught Logic: 0]
736401854 False 3/10 [Time: 0.016, Eff: 2000%, Risk: 20.59%, Guesses: 1, Uncaught Logic: 0]
2733015921 True 4/11 [Time: 3.890, Eff: 6%, Risk: 15.22%, Guesses: 3, Uncaught Logic: 0]
1251519133 True 5/